In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [11]:
# Data pipeline

transform = transforms.Compose([
    transforms.ToTensor(), # Converts images to tensors and scales pixels to range 0-1
    transforms.Normalize((0.1307,), (0.3081,)) # Shifts and scales to be centered around 0
])

In [12]:
# Load MNIST datasets

train_dataset = torchvision.datasets.MNIST(
    root = "./data", train = True, download = True, transform = transform
)

test_dataset = torchvision.datasets.MNIST(
    root = "./data", train = False, download = True, transform = transform
)

In [13]:
# Create data loaders

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 1000, shuffle = False)

In [14]:
# Create neural network

class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.layers(x)
        return x


In [15]:
# Device management and optimizer

# Check for GPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device}")

# Initialize model and move to GPU device
model = MNISTClassifier().to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Using mps


In [16]:
# Define training stage

def train_epoch(model, train_loader, loss_function, optimizer, device):
    model.train() # Puts model into train mode

    # Tracking variables
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data ,target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        # Track progress
        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Print every 100 batches
        if batch_idx % 100 == 0 and batch_idx > 0:
            avg_loss = running_loss / 100
            accuracy = 100. * (correct / total)
            print(f"[{batch_idx * 64}/{60000}]"
                  f"Loss: {avg_loss:.3f} | Accuracy: {accuracy:.1f}%")
            running_loss = 0.0

In [17]:
# Define evaluation stage 

def evaluate(model, test_loader, device):
    model.eval() # Puts model into evaluation mode

    # Tracking variables
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return 100. * (correct / total)
    

In [18]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f"\nEpoch: {epoch + 1}")
    train_epoch(model, train_loader, loss_function, optimizer, device)
    accuracy = evaluate(model, test_loader, device)
    print(f"Test accuracy: {accuracy:.2f}%")


Epoch: 1
[6400/60000]Loss: 0.623 | Accuracy: 81.9%
[12800/60000]Loss: 0.327 | Accuracy: 86.0%
[19200/60000]Loss: 0.272 | Accuracy: 88.0%
[25600/60000]Loss: 0.242 | Accuracy: 89.2%
[32000/60000]Loss: 0.205 | Accuracy: 90.1%
[38400/60000]Loss: 0.179 | Accuracy: 90.9%
[44800/60000]Loss: 0.170 | Accuracy: 91.4%
[51200/60000]Loss: 0.166 | Accuracy: 91.8%
[57600/60000]Loss: 0.144 | Accuracy: 92.3%
Test accuracy: 95.69%

Epoch: 2
[6400/60000]Loss: 0.126 | Accuracy: 96.3%
[12800/60000]Loss: 0.125 | Accuracy: 96.4%
[19200/60000]Loss: 0.120 | Accuracy: 96.4%
[25600/60000]Loss: 0.111 | Accuracy: 96.5%
[32000/60000]Loss: 0.115 | Accuracy: 96.4%
[38400/60000]Loss: 0.097 | Accuracy: 96.6%
[44800/60000]Loss: 0.108 | Accuracy: 96.6%
[51200/60000]Loss: 0.106 | Accuracy: 96.6%
[57600/60000]Loss: 0.095 | Accuracy: 96.7%
Test accuracy: 97.24%

Epoch: 3
[6400/60000]Loss: 0.080 | Accuracy: 97.8%
[12800/60000]Loss: 0.082 | Accuracy: 97.7%
[19200/60000]Loss: 0.074 | Accuracy: 97.7%
[25600/60000]Loss: 0.074 |